In [1]:
from Health_chat import HealthChat
import torch
import os
import sys

c:\Users\nibaaa\.conda\envs\dia\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
myHealthChat = HealthChat()


✅ Tokenizer is loaded successfully: /tokenizer/tokenizerForHealthCare.pkl
✅ Tokenizer is loaded successfully: /tokenizer/tokenizerForMentalHealth.pkl


In [3]:
input = "i feel like i have persistent knee pain that has been going on for two days."

input2 = "I get tired when I breath. Like, when I breath, my heart starts to beat so fast. I m 15 years old, and from time to time I smoke marijuana, but this happened to me even before I started to smoke marijuana. Down between my neck (throat) and upper chests, I can feel like some type of pressure, and sometimes even when I move from one side to the other side, my heart starts to beat so fast and I get this weird pressure in my chest/between my chest and throat/neck."
print(myHealthChat.chat(input))


myHealthChat.set_current_model(2)
print(myHealthChat.chat(input))

myHealthChat.set_current_model(3)
print(myHealthChat.chat(input))

myHealthChat.set_current_model(4)
print(myHealthChat.chat(input))


('. to on the query . i can that you have is through . <UNKNOWN> . can be to is infected is the its . its . as well you you have taken through the and its . then with symptoms then treatment is treated time treatment treatment such such . consult and avoid treatment . you not . that such can still treatment treatment such an oral and treatment hope the is also that you .', 'physical', 'biLSTM')
('hi welcome to chat doctor . i studied your query and pain is most herniated neck pain neck pain is suggestive of musculoskeletal pain neck pain neck pain is suggestive of neck pain is suggestive of neck pain increases increasing neck pain increases increasing neck pain and would be confirmed ct scan further .', 'physical', 'biGRU')
('iin loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss loss 

In [9]:
# mental health
input = "I hate my life and I feel like I am not worth it. I am so sad and I don't know what to do."
print(myHealthChat.match_intent_keyWorld(myHealthChat.clean_text(input)))
print(myHealthChat.match_intent_CosineSim(myHealthChat.clean_text(input)))
print(myHealthChat.chat(input))


psychological_agent
psychological_agent
('often slowly up signs from too out <UNKNOWN> or or or or illusion off or past so hurtful bc unusual or besides embarrassment back mentally posttraumatic or se lot , deep discipline off him social social return impossible . other strong no his new another even past or or or or or or his his or their as the the its per accomplish all in past or or or or its passed passed passed too too too , strong his strong strong breath no all all abusive or or or even is absolutely it was try try is distress did', 'mental', 'transformerV2h')


In [10]:
print(myHealthChat.get_current_intent_matching_method())
myHealthChat.set_intent_matching_method(2, 30)
print(myHealthChat.get_current_intent_matching_method())

(1, 'keyWorld', 16)
(2, 'CosineSim', 30)


In [11]:
import pandas as pd

TESTING_NUM = 15

# get input training data
df_physiological = pd.read_parquet("utils/healthCare.parquet").dropna()
df_psychological = pd.read_json("utils/mentalHealth.json", lines=True).dropna()

df_physiological = df_physiological.sample(TESTING_NUM, random_state=45)
df_psychological = df_psychological.sample(TESTING_NUM, random_state=45)

# Testing 1: intent matching

In [12]:
import time

myHealthChat.get_current_intent_matching_method()
# plan to do this by calculating the confusion matrix and average response time

# method 1: keyword matching
# method 2: embedding + cosine similarity
# training_num: 16, 30, 50, this is for number of data for embedding training
inputParam = [{"intentMatchingMethod": 1, "intent_training_num":16}, 
                {"intentMatchingMethod": 2, "intent_training_num":16}, 
				{"intentMatchingMethod": 2, "intent_training_num":30}, 
				{"intentMatchingMethod": 2, "intent_training_num":50}]

for i in range(len(inputParam)):
	tp = 0
	fp = 0
	fn = 0
	tn = 0

	myHealthChat.set_intent_matching_method(inputParam[i]["intentMatchingMethod"], inputParam[i]["intent_training_num"])

	print("==========================")

	if inputParam[i]["intentMatchingMethod"] == 1:
		print("Intent matching method: keyword matching")
	else:
		print("Intent matching method: embedding + cosine similarity")
		print("Training data number: ", inputParam[i]["intent_training_num"])

	# test physiological intent matching
	# there input are all physiological, so we can use this to test the true positive rate
	time_start = time.time()
	for index, row in df_physiological.iterrows(): 
		input = row["input"]
		input = myHealthChat.clean_text(input)
		if inputParam[i]["intentMatchingMethod"] == 1:
			cat = myHealthChat.match_intent_keyWorld(input)
		else:
			cat = myHealthChat.match_intent_CosineSim(input)
		if cat == "physiological_agent":  
			tp += 1  # true positive
		else:  
			fn += 1  # false negative


	# test psychological intent matching
	# there input are all psychological, so we can use this to test the true negative rate
	for index, row in df_psychological.iterrows():
		input = row["Context"]
		input = myHealthChat.clean_text(input)
		if inputParam[i]["intentMatchingMethod"] == 1:
			cat = myHealthChat.match_intent_keyWorld(input)
		else:
			cat = myHealthChat.match_intent_CosineSim(input)
		if cat == "psychological_agent":  
			tn += 1  # true negative
		else:  
			fp += 1  # false positive
	time_end = time.time()
	total_time = time_end - time_start
	avg_time = total_time / (TESTING_NUM*2) * 1000  # convert to ms
	# calculate the classification accuracy
	accuracy = (tp + tn) / (tp + tn + fp + fn)
	print(myHealthChat.get_current_intent_matching_method())
	print("accuracy: ", accuracy)
	print("average response time: ", avg_time, "ms")

    

Intent matching method: keyword matching
(1, 'keyWorld', 16)
accuracy:  0.5333333333333333
average response time:  0.1070261001586914 ms
Intent matching method: embedding + cosine similarity
Training data number:  16
(2, 'CosineSim', 16)
accuracy:  0.9666666666666667
average response time:  5.71138064066569 ms
Intent matching method: embedding + cosine similarity
Training data number:  30
(2, 'CosineSim', 30)
accuracy:  0.9333333333333333
average response time:  6.67890707651774 ms
Intent matching method: embedding + cosine similarity
Training data number:  50
(2, 'CosineSim', 50)
accuracy:  0.9666666666666667
average response time:  8.179918924967447 ms


# Testing 2: response method testing

In [15]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# 确保 NLTK 数据包已安装
nltk.download('punkt', quiet=True)

# 计算 BLEU 分数的函数
def calculate_bleu(candidate: str, reference: str) -> float:
    # 分词
    candidate_tokens = nltk.word_tokenize(candidate)
    reference_tokens = nltk.word_tokenize(reference)

    # 将参考文本包装为列表（nltk 要求的格式）
    reference_list = [reference_tokens]

    # 使用 SmoothingFunction 避免零分问题
    smoothing = SmoothingFunction().method1

    # 计算 BLEU 分数（1-gram 到 4-gram）
    bleu_score = sentence_bleu(reference_list, candidate_tokens, 
                               weights=(0.25, 0.25, 0.25, 0.25), 
                               smoothing_function=smoothing)

    return bleu_score

candidate_text = "To gain weight with IBS, focus on small, frequent meals, nutrient-dense foods, low-FODMAP options, protein-rich sources, and digestive enzyme support."
reference_text = "it can be a cause of epilepsy is a sign of a flu shot you may not be a cure for a pregnancy test and they are known i am a day you may need to know for a pregnancy test and they are known i am a day you may need to know for a pregnancy test and they are known i am."
score = calculate_bleu(candidate_text, reference_text)
print(f"BLEU Score: {score:.4f}")


BLEU Score: 0.0020


In [4]:
import pandas as pd

TESTING_NUM_GENERATE = 15

# get input training data
df_physiological = pd.read_parquet("utils/healthCare.parquet").dropna()

mid_index_phy = len(df_physiological) // 2

# df_physiological = df_physiological.iloc[mid_index_phy:].reset_index(drop=True)

df_physiological = df_physiological[df_physiological['input'].apply(lambda x: len(str(x).split()) <= 120)]

df_filtered_physiological = df_physiological[df_physiological['output'].apply(lambda x: len(str(x).split()) <= 150)]

df_filtered_physiological = df_filtered_physiological.sample(TESTING_NUM_GENERATE, random_state=28)

df_filtered_physiological.head()

,instruction,input,output
61959,"If you are a doctor, please answer the medical...","Hi my name is Mandilakhe, i have been sufferin...",Degree understand your concerns went through y...
21505,"If you are a doctor, please answer the medical...","Hi , this is Manjot, I have brown skin and I w...",Thank you for query. Natural color is the best...
65807,"If you are a doctor, please answer the medical...",I feel a tightness in my upper chest when I be...,Thanks for your question on Chat Doctor. In my...
5201,"If you are a doctor, please answer the medical...","i am male 29, i got itch around my scrotum and...","HelloSign, how many days, you are having this ..."
43588,"If you are a doctor, please answer the medical...",hi xyz im 29 my penis size is 4inches when it ...,"Hello dear, The size of penis is genetically d..."


In [5]:
df = pd.read_json("utils/testing.json").dropna()

df.head()

,input,output
0,I guess if its free I have been walking 6 mile...,"Hello, The symptoms seem to be related to peri..."
1,Hi just waiting on mri scan but neurologist wo...,"Hi, Thank you for posting your query. Your sym..."
2,I am 49 years old. I have been getting pains s...,"Hello, I have studied your case. I will advise..."
3,I have been experiencing a great amount of low...,Dear patient This seems high velocity back tra...
4,I have been experiencing frequent headaches an...,"Hello, it seems you may be experiencing tensio..."


In [ ]:

import time
# myHealthChat.get_current_model()
# myHealthChat.set_current_model()

# model_id: 1 for biLSTM, 2 for biGRU, 3 for transformerV1, 4 for transformerV2
inputParam = [
			{"modelId": 1, "modelWeightPath": "checkpoint/weight_biLSTM_100.pth"}, 
			{"modelId": 1, "modelWeightPath":"checkpoint/weight_biLSTM_550.pth"}, 
			{"modelId": 2, "modelWeightPath":'checkpoint/weight_biGRU_100.pth'}, 
			{"modelId": 2, "modelWeightPath":'checkpoint/weight_biGRU_550.pth'},
			{"modelId": 2, "modelWeightPath":'checkpoint/weight_biGRU_1550.pth'},
			{"modelId": 3, "modelWeightPath":'checkpoint/weight_transformer_550.pth'},
			{"modelId": 3, "modelWeightPath":'checkpoint/weight_transformer_1550.pth'},
			{"modelId": 3, "modelWeightPath":'checkpoint/weight_transformer_3550.pth'},
			{"modelId": 4, "modelWeightPath":'checkpoint/weight_transformerV2_550.pth'},
			{"modelId": 4, "modelWeightPath":'checkpoint/weight_transformerV2_1550.pth'},
			{"modelId": 4, "modelWeightPath":'checkpoint/weight_transformerV2_3550.pth'}
			]


for i in range(len(inputParam)):
	print("=============================")
	myHealthChat.set_current_model(inputParam[i]["modelId"])
	print(myHealthChat.get_current_model())
	myHealthChat.set_model_weight(inputParam[i]["modelId"] ,inputParam[i]["modelWeightPath"])
	mid, mname = myHealthChat.get_current_model()
	print(f"model: {mname}")
	weight = inputParam[i]["modelWeightPath"]
	print(f"model weight: {weight}")
	total_score = 0
	total_time = 0
	
	for index, row in df.iterrows(): 
		input = row["input"]
		input = myHealthChat.clean_text(input)
		time_start = time.time()
		output, intent, modelG = myHealthChat.get_model_response_physical(input)  # 运行到这里报错
		reference_text = myHealthChat.clean_text(row["output"])
		time_end = time.time()
		total_time = time_end - time_start
		blueScore = calculate_bleu(candidate=output, reference=reference_text)
		total_score = total_score+blueScore

	
	avg_time = total_time / TESTING_NUM_GENERATE * 1000  # ms
	avg_blueScore = blueScore / TESTING_NUM_GENERATE

	print(f"avg BLEU Score: {avg_blueScore:.4f}")
	print(f"avg respons time: {avg_time:.4f}")
	print(f"total respons time: {total_time:.4f}")


(1, 'biLSTM')
model: biLSTM
model weight: checkpoint/weight_biLSTM_100.pth
avg BLEU Score: 0.0047
avg respons time: 3.9033
total respons time: 0.0585
(1, 'biLSTM')
model: biLSTM
model weight: checkpoint/weight_biLSTM_550.pth
avg BLEU Score: 0.0046
avg respons time: 3.9052
total respons time: 0.0586
(2, 'biGRU')
model: biGRU
model weight: checkpoint/weight_biGRU_100.pth
avg BLEU Score: 0.0098
avg respons time: 2.2279
total respons time: 0.0334
(2, 'biGRU')
model: biGRU
model weight: checkpoint/weight_biGRU_550.pth
avg BLEU Score: 0.0069
avg respons time: 3.6769
total respons time: 0.0552
(2, 'biGRU')
model: biGRU
model weight: checkpoint/weight_biGRU_1550.pth
avg BLEU Score: 0.0097
avg respons time: 2.7462
total respons time: 0.0412
(3, 'transformerV1')
model: transformerV1
model weight: checkpoint/weight_transformer_550.pth
avg BLEU Score: 0.0007
avg respons time: 15.2846
total respons time: 0.2293
(3, 'transformerV1')
model: transformerV1
model weight: checkpoint/weight_transformer_15